# 03) 구글 BERT의 마스크드 언어 모델(Masked Language Model) 실습

## 1. 마스크드 언어 모델과 토크나이저

In [1]:
# 모델과 토크나이저를 로드
# 사전 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑관계
# 맵핑 정수 체계가 같아야 한다.
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [2]:
# BERT를 마스크드 언어 모델 형태로 로드
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
# 해당 모델이 학습되었을 당시에 사용되었던 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

## 2. BERT의 입력

In [6]:
# 마스킹된 시퀀스를 예측하기 위해서 정수 인코딩
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [7]:
# input_ids를 통해 정수 인코딩 결과를 확인할 수 있다.
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [9]:
# 'token_type_ids'를 통해 문장 세그먼트 인코딩 결과 확인
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스를 얻습니다. 만약 문장이 두 개였다면 두번째 문장이 시작되는 구간부터는 1의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다.

토크나이저로 변환된 결과에서 attention_mask를 통해서 실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크를 확인할 수 있습니다.

In [10]:
# 현재 입력에는 패딩이없으므로 문장의 길이만큼 1 시퀀스를 얻게 된다.
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


## 3. [MASK] 토큰 예측하기

FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줍니다. FillMaskPipeline에 우선 앞서 불러온 모델과 토크나이저를 지정해줍니다.

In [11]:
# BERT 예측 출력값 정리해서 보여주는 라이브러리 로드
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [12]:
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621117234230042,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20342008769512177,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208536267280579,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.001863025827333331,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354872353374958,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [13]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.25628945231437683,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284120619297028,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107694357633591,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248982042074203,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046642154455185,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [14]:
pip('I went to [MASK] this morning.')

[{'score': 0.357307493686676,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304452002048492,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.12845061719417572,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.06230563297867775,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.04695260524749756,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]

In [15]:
inputs[]

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[ 101, 4715, 2003, 1037, 2428, 4569,  103, 1012,  102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]])>}

In [17]:
pip("[MASK] is boring.")

[{'score': 0.19113168120384216,
  'token': 2166,
  'token_str': 'life',
  'sequence': 'life is boring.'},
 {'score': 0.16215674579143524,
  'token': 2023,
  'token_str': 'this',
  'sequence': 'this is boring.'},
 {'score': 0.1452539712190628,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'school is boring.'},
 {'score': 0.10120807588100433,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'it is boring.'},
 {'score': 0.028992420062422752,
  'token': 2673,
  'token_str': 'everything',
  'sequence': 'everything is boring.'}]